In [12]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/haarcascade-frontalface/haarcascade_frontalface_default.xml
/kaggle/input/facial-expression-recognitionferchallenge/Submission.csv
/kaggle/input/facial-expression-recognitionferchallenge/fer2013/fer2013/README
/kaggle/input/facial-expression-recognitionferchallenge/fer2013/fer2013/fer2013.csv
/kaggle/input/facial-expression-recognitionferchallenge/fer2013/fer2013/fer2013.bib


In [2]:
import sys, os
import pandas as pd
import numpy as np

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization,AveragePooling2D
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.utils import np_utils

In [3]:


data = pd.read_csv("/kaggle/input/facial-expression-recognitionferchallenge/fer2013/fer2013/fer2013.csv")

In [4]:
X_train,train_y,X_test,test_y=[],[],[],[]

for index, row in data.iterrows():
    val=row['pixels'].split(" ")
    try:
        if 'Training' in row['Usage']:
           X_train.append(np.array(val,'float32'))
           train_y.append(row['emotion'])
        elif 'PublicTest' in row['Usage']:
           X_test.append(np.array(val,'float32'))
           test_y.append(row['emotion'])
    except:
        print(f"error occured at index :{index} and row:{row}")


num_features = 64
num_labels = 7
batch_size = 64
epochs = 200
width, height = 48, 48


X_train = np.array(X_train,'float32')
train_y = np.array(train_y,'float32')
X_test = np.array(X_test,'float32')
test_y = np.array(test_y,'float32')

train_y=np_utils.to_categorical(train_y, num_classes=num_labels)
test_y=np_utils.to_categorical(test_y, num_classes=num_labels)

#cannot produce
#normalizing data between oand 1
X_train -= np.mean(X_train, axis=0)
X_train /= np.std(X_train, axis=0)

X_test -= np.mean(X_test, axis=0)
X_test /= np.std(X_test, axis=0)

X_train = X_train.reshape(X_train.shape[0], 48, 48, 1)

X_test = X_test.reshape(X_test.shape[0], 48, 48, 1)

# print(f"shape:{X_train.shape}")
##designing the cnn
#1st convolution layer
model = Sequential()

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=(X_train.shape[1:])))
model.add(Conv2D(64,kernel_size= (3, 3), activation='relu'))
# model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))
model.add(Dropout(0.5))

#2nd convolution layer
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
# model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))
model.add(Dropout(0.5))

#3rd convolution layer
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Conv2D(128, (3, 3), activation='relu'))
# model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))

model.add(Flatten())

#fully connected neural networks
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(num_labels, activation='softmax'))

# model.summary()

#Compliling the model
model.compile(loss=categorical_crossentropy,
              optimizer=Adam(),
              metrics=['accuracy'])

#Training the model
model.fit(X_train, train_y,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, test_y),
          shuffle=True)


Epoch 1/200
449/449 [==============================] - 6s 14ms/step - loss: 1.7156 - accuracy: 0.2975 - val_loss: 1.6297 - val_accuracy: 0.3536
Epoch 2/200
449/449 [==============================] - 6s 13ms/step - loss: 1.5208 - accuracy: 0.4018 - val_loss: 1.3936 - val_accuracy: 0.4558
Epoch 3/200
449/449 [==============================] - 6s 12ms/step - loss: 1.4125 - accuracy: 0.4515 - val_loss: 1.3114 - val_accuracy: 0.4887
Epoch 4/200
449/449 [==============================] - 6s 12ms/step - loss: 1.3443 - accuracy: 0.4804 - val_loss: 1.2644 - val_accuracy: 0.5166
Epoch 5/200
449/449 [==============================] - 6s 12ms/step - loss: 1.2958 - accuracy: 0.4993 - val_loss: 1.2707 - val_accuracy: 0.5049
Epoch 6/200
449/449 [==============================] - 6s 12ms/step - loss: 1.2586 - accuracy: 0.5188 - val_loss: 1.2151 - val_accuracy: 0.5308
Epoch 7/200
449/449 [==============================] - 6s 12ms/step - loss: 1.2241 - accuracy: 0.5306 - val_loss: 1.2159 - val_accuracy:

449/449 [==============================] - 6s 12ms/step - loss: 0.5532 - accuracy: 0.7959 - val_loss: 1.4618 - val_accuracy: 0.5784
Epoch 58/200
449/449 [==============================] - 6s 12ms/step - loss: 0.5507 - accuracy: 0.7988 - val_loss: 1.4273 - val_accuracy: 0.5729
Epoch 59/200
449/449 [==============================] - 6s 12ms/step - loss: 0.5485 - accuracy: 0.8000 - val_loss: 1.4110 - val_accuracy: 0.5818
Epoch 60/200
449/449 [==============================] - 6s 12ms/step - loss: 0.5423 - accuracy: 0.7991 - val_loss: 1.4823 - val_accuracy: 0.5729
Epoch 61/200
449/449 [==============================] - 6s 12ms/step - loss: 0.5383 - accuracy: 0.8036 - val_loss: 1.4311 - val_accuracy: 0.5790
Epoch 62/200
449/449 [==============================] - 6s 12ms/step - loss: 0.5291 - accuracy: 0.8050 - val_loss: 1.4583 - val_accuracy: 0.5832
Epoch 63/200
449/449 [==============================] - 5s 12ms/step - loss: 0.5277 - accuracy: 0.8098 - val_loss: 1.4499 - val_accuracy: 0.582

449/449 [==============================] - 6s 12ms/step - loss: 0.3813 - accuracy: 0.8667 - val_loss: 1.6439 - val_accuracy: 0.5698
Epoch 114/200
449/449 [==============================] - 5s 12ms/step - loss: 0.3726 - accuracy: 0.8694 - val_loss: 1.6955 - val_accuracy: 0.5759
Epoch 115/200
449/449 [==============================] - 6s 12ms/step - loss: 0.3625 - accuracy: 0.8758 - val_loss: 1.6888 - val_accuracy: 0.5795
Epoch 116/200
449/449 [==============================] - 6s 12ms/step - loss: 0.3594 - accuracy: 0.8751 - val_loss: 1.7989 - val_accuracy: 0.5690
Epoch 117/200
449/449 [==============================] - 6s 13ms/step - loss: 0.3709 - accuracy: 0.8725 - val_loss: 1.7555 - val_accuracy: 0.5645
Epoch 118/200
449/449 [==============================] - 6s 12ms/step - loss: 0.3655 - accuracy: 0.8735 - val_loss: 1.7687 - val_accuracy: 0.5762
Epoch 119/200
449/449 [==============================] - 6s 13ms/step - loss: 0.3657 - accuracy: 0.8725 - val_loss: 1.7433 - val_accuracy:

449/449 [==============================] - 6s 12ms/step - loss: 0.3042 - accuracy: 0.8978 - val_loss: 1.8514 - val_accuracy: 0.5743
Epoch 170/200
449/449 [==============================] - 6s 12ms/step - loss: 0.3057 - accuracy: 0.8973 - val_loss: 1.8807 - val_accuracy: 0.5653
Epoch 171/200
449/449 [==============================] - 6s 13ms/step - loss: 0.3037 - accuracy: 0.8961 - val_loss: 1.9578 - val_accuracy: 0.5681
Epoch 172/200
449/449 [==============================] - 6s 13ms/step - loss: 0.3182 - accuracy: 0.8948 - val_loss: 1.8882 - val_accuracy: 0.5706
Epoch 173/200
449/449 [==============================] - 6s 13ms/step - loss: 0.3134 - accuracy: 0.8956 - val_loss: 1.8248 - val_accuracy: 0.5804
Epoch 174/200
449/449 [==============================] - 6s 12ms/step - loss: 0.3108 - accuracy: 0.8957 - val_loss: 2.0194 - val_accuracy: 0.5776
Epoch 175/200
449/449 [==============================] - 6s 12ms/step - loss: 0.3001 - accuracy: 0.8980 - val_loss: 1.9269 - val_accuracy:

In [10]:
fer_json = model.to_json()
with open("fer.json", "w") as json_file:
    json_file.write(fer_json)
model.save_weights("fer.h5")